In [37]:
import numpy as np
import pandas as pd
# neither year nor len are statistically significant predictors of sex
from pathlib import Path
import numpy as np
import pandas as pd

In [38]:
CWD = Path('.').absolute().resolve()
DATA_DIR = '.nlpia2-data'
DATA_FILE = 'baby-names-region.csv.gz'
CWD

PosixPath('/home/hobs/code/tangibleai/nlpia2/src/nlpia2/ch05')

In [39]:
parent = CWD
data_dir = parent / DATA_DIR 
filepath = data_dir / DATA_FILE
for i in range(10):
    print(filepath)
    if filepath.is_file():
        break
    parent = parent.parent
    data_dir = parent / DATA_DIR 
    filepath = data_dir / DATA_FILE
filepath

/home/hobs/code/tangibleai/nlpia2/src/nlpia2/ch05/.nlpia2-data/baby-names-region.csv.gz
/home/hobs/code/tangibleai/nlpia2/src/nlpia2/.nlpia2-data/baby-names-region.csv.gz
/home/hobs/code/tangibleai/nlpia2/src/.nlpia2-data/baby-names-region.csv.gz
/home/hobs/code/tangibleai/nlpia2/.nlpia2-data/baby-names-region.csv.gz


PosixPath('/home/hobs/code/tangibleai/nlpia2/.nlpia2-data/baby-names-region.csv.gz')

In [40]:
df = pd.read_csv(filepath)

In [41]:
np.random.seed(451)
df = df.sample(10_000)
df.head()

,region,sex,year,name,count,freq
6139665,WV,F,1987,Brittani,10,0.000003
2565339,MD,F,1954,Ida,18,0.000005
22297,AK,M,1988,Maxwell,5,0.000001
5114650,TN,F,1972,Charlene,24,0.000008
2126395,KS,M,1954,Todd,11,0.000003


In [42]:
names = df['name'].unique()
names[:10]

array(['Brittani', 'Ida', 'Maxwell', 'Charlene', 'Todd', 'Aubrey',
       'Arianna', 'Otis', 'Trenton', 'Faustino'], dtype=object)

In [43]:
len(names) / len(df)

0.4025

In [ ]:
# df = pd.get_dummies(df, columns=['region'])
# df.head()

In [44]:
df = df.groupby(['name', 'sex']).mean()
df.head()

,,year,count,freq
name,sex,,,
Aaden,M,2008.0,51.0,0.000015
Aahana,F,2018.0,26.0,0.000009
Aahil,M,2019.0,5.0,0.000002
Aaleyah,F,2010.0,17.0,0.000005
Aalia,F,2016.5,6.5,0.000002


In [57]:
df['name'] = df.index.get_level_values('name')
df['sex'] = df.index.get_level_values('sex')
df.head()

,,year,count,freq,name,sex
name,sex,,,,,
Aaden,M,2008.0,51.0,0.000015,Aaden,M
Aahana,F,2018.0,26.0,0.000009,Aahana,F
Aahil,M,2019.0,5.0,0.000002,Aahil,M
Aaleyah,F,2010.0,17.0,0.000005,Aaleyah,F
Aalia,F,2016.5,6.5,0.000002,Aalia,F


In [61]:
df.query('name == "Chris"')


year  count      freq   name sex
name  sex                                    
Chris F    1983.0   5.00  0.000002  Chris   F
      M    1962.5  59.75  0.000017  Chris   M

In [63]:
df.loc[pd.IndexSlice['Chris', :]]

,year,count,freq,name,sex
sex,,,,,
F,1983.0,5.00,0.000002,Chris,F
M,1962.5,59.75,0.000017,Chris,M


In [65]:
df['istrain'] = np.random.rand(len(df)) < .9
df.head()

,,year,count,freq,name,sex,istrain
name,sex,,,,,,
Aaden,M,2008.0,51.0,0.000015,Aaden,M,True
Aahana,F,2018.0,26.0,0.000009,Aahana,F,True
Aahil,M,2019.0,5.0,0.000002,Aahil,M,True
Aaleyah,F,2010.0,17.0,0.000005,Aaleyah,F,False
Aalia,F,2016.5,6.5,0.000002,Aalia,F,True


In [66]:
istrain = df['istrain']
del df['istrain']
istrain.sum() / len(istrain)

0.8966493629070316

In [67]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced')
model

LogisticRegression(class_weight='balanced')

In [75]:
# df['name'] = df.index.values
df['len'] = df['name'].str.len()
X = df[['len']]
y = df['sex']
X.head()

,,len
name,sex,
Aaden,M,5
Aahana,F,6
Aahil,M,5
Aaleyah,F,7
Aalia,F,5


In [76]:
model.fit(X[istrain], y[istrain], sample_weight=df['freq'][istrain])

LogisticRegression(class_weight='balanced')

In [77]:
model.score(X[istrain], y[istrain], sample_weight=df['freq'][istrain])

0.448398409763099

In [78]:
model.score(X[~istrain], y[~istrain], sample_weight=df['freq'][~istrain])

0.510545746795573

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3), lowercase=False)
vectorizer

TfidfVectorizer(analyzer='char', lowercase=False, ngram_range=(1, 3))

In [83]:
vectorizer.fit(df['name'][istrain])

TfidfVectorizer(analyzer='char', lowercase=False, ngram_range=(1, 3))

In [87]:
vecs = vectorizer.transform(df['name'])
vecs = pd.DataFrame.sparse.from_spmatrix(vecs)
vecs.head()


,0,1,2,3,4,5,6,7,8,9,...,3666,3667,3668,3669,3670,3671,3672,3673,3674,3675
0,0.195328,0.401890,0.508442,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.183625,0.377812,0.000000,0.455313,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.186853,0.384453,0.000000,0.463316,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.165018,0.339527,0.000000,0.000000,0.394721,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.213283,0.438833,0.000000,0.000000,0.510171,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
vecs.columns = vectorizer.get_feature_names_out()
vecs.index = df.index
vecs.head()[vecs.columns[:5]]

,,A,Aa,Aad,Aah,Aal
name,sex,,,,,
Aaden,M,0.195328,0.401890,0.508442,0.000000,0.000000
Aahana,F,0.183625,0.377812,0.000000,0.455313,0.000000
Aahil,M,0.186853,0.384453,0.000000,0.463316,0.000000
Aaleyah,F,0.165018,0.339527,0.000000,0.000000,0.394721
Aalia,F,0.213283,0.438833,0.000000,0.000000,0.510171


In [93]:
model.fit(vecs[istrain], y[istrain], sample_weight=df['freq'][istrain])

LogisticRegression(class_weight='balanced')

In [96]:
model.score(vecs[~istrain], y[~istrain], sample_weight=df['freq'][~istrain])

0.510545746795573

In [97]:
model.classes_


array(['F', 'M'], dtype=object)

In [101]:
model.predict_proba(vectorizer.transform(['Kemal', 'Lane', 'Hobson', 'Vish']))

array([[0.46525322, 0.53474678],
       [0.46525837, 0.53474163],
       [0.46520632, 0.53479368],
       [0.46526069, 0.53473931]])

In [102]:
model.predict_proba(vectorizer.transform(['Maria', 'Syndee', 'Hobson']))

array([[0.46529297, 0.53470703],
       [0.46524354, 0.53475646],
       [0.46520632, 0.53479368]])